# Clustering

In [1]:
from tqdm import tqdm_notebook, tqdm 
import pandas as pd 
from functions.others import *
from functions.cluster import * 
import os 

from TrackML.dataset import load_event
from TrackML.randomize import shuffle_hits
from TrackML.score import score_event


In [2]:
weights={'pi':1,'theta':0.15}
stds={'z0':7.5, 'kt':7.5e-4}
d =    {'sint':[225,110,110],
        'cost':[225,110,110],
          'phi':[550,260,260],
        'min_group':[11,11,10],
        'npoints':[250,1250,500]}
filters=pd.DataFrame(d)
filters

,sint,cost,phi,min_group,npoints
0,225,225,550,11,250
1,110,110,260,11,1250
2,110,110,260,10,500


In [3]:
event_prefix = 'event000001000'


def run_clustering(i):
    event_num=1000+i
    event_prefix = 'event00000{}'.format(event_num)
    print('running '+event_prefix)
    hits, cells, particles, truth = load_event(os.path.join('data/train_100_events', event_prefix))
    history=[]

    resa1=clustering(hits,stds,filters)
    resa1["event_id"]=event_num-1000
    score = score_event_fast(truth, resa1.rename(index=str, columns={"label": "track_id"}))
    print("Your score: ", score)
    resa1.to_csv("{}{}_cluster.csv".format('clustering/',event_prefix), index=False)

In [4]:
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm

num_cores = multiprocessing.cpu_count() - 2
num_cores

6

In [5]:
res_list = Parallel(n_jobs=num_cores)(delayed(run_clustering)(i) for i in tqdm(range(100)))

100%|██████████| 100/100 [46:04<00:00, 27.64s/it]
